In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
#from sklearn.preprocessing import MinMaxScaler
from kmodes.kmodes import KModes
from scipy import stats

In [ ]:
#df=pd.read_csv('Fala-nos mais sobre ti R_S (Respostas)_csv.csv')

In [ ]:
before=pd.read_excel('Fala-nos mais sobre ti R_S (Respostas).xlsx')

In [ ]:
rise=pd.read_excel('Feedback Rise (Respostas).xlsx')

In [ ]:
shine=pd.read_excel('Feedback Shine (Respostas).xlsx')

In [ ]:
rel_impact=pd.read_excel('Relatório de Impacto (Respostas).xlsx')

In [ ]:
print(f'Number of answers before : {before.shape[0]}')
print(f'Number of answers rise : {rise.shape[0]}') 
print(f'Number of answers shine : {shine.shape[0]}') 
print(f'Number of answers impact : {rel_impact.shape[0]}') 

## Ficheiro "Fala-nos mais sobre ti" (before)

In [ ]:
X=before.copy()

In [ ]:
X.shape

In [ ]:
# Lets rename columns
new_names= {"Já conhecias a GirlMOVE Academy?":"Já conhecia a GirlMOVE Academy",
            "Porque te inscreveste no Rise&Shine? O que esperas aprender?":"Razão da inscrição e expectativas",
            "Qual é a tua ocupação atual?":"Ocupação Actual",
            "Qual é a tua ocupação atual?.1":"Ocupação Actual_Descrição",
            "Quais são para ti os próximos passos?":"Próximos Passos",
            "Qual é o sítio de sonho onde gostarias de trabalhar e porquê?":"Trabalho de sonho",
            "Qual é a tua maior ambição?": "Maior ambição",
            "Se só pudesses escolher uma coisa para fazeres o resto da tua vida o que escolherias?":"Apenas uma coisa para fazer durante o resto da vida",
            "Ao proceder ao preenchimento do formulário supra, tomei conhecimento de que os meus dados serão tratados pela Girl MOVE Academy, com sede na Av. Defensores de Chaves, 69, R/C Esq, Lisboa, e que esses dados serão tratados no âmbito desta iniciativa. Tomei igualmente conhecimento dos meus direitos de acesso, retificação, apagamento, limitação, oposição ou portabilidade relativamente aos meus dados pessoais, direitos que poderão ser exercidos por mim mediante o envio de um correio eletrónico para riseandshine@girlmove.org. ":"Tratamento de dados"
           }
X.columns=X.columns.map(new_names)
#df.set_axis(['#new names= {'Já conhecias a GirlMOVE Academy?':}', 'Porque te inscreveste e que esperas aprender?','Ocupação Actual', 'Ocupação Actual_Descrição','Próximos Passos',''])
#new names= {'Já conhecias a GirlMOVE Academy?':}

In [ ]:
X.head(2)

## Estatísticas dos dados

In [ ]:
X.dtypes

In [ ]:
X.nunique().sort_values(ascending=False)

In [ ]:
sns.countplot(X['Tratamento de dados']);

In [ ]:
X.isnull().sum()

#### - There's no missing values, good! We don't have to worry about that

In [ ]:
X['Já conhecia a GirlMOVE Academy'].value_counts()

In [ ]:
sns.countplot(X['Já conhecia a GirlMOVE Academy']);

In [ ]:
X.describe()

#### - The table describe shows that the most frequent "Ocupação Actual_Descrição" is "Desempregada" with 11. However in the excel file there are 12 "Desempregadas".
#### - Lets find out what happened

In [ ]:
# O CÓDIGO SEGUINTE NÃO É NECESSÁRIO, É APENAS UMA CURIOSIDADE
# AO APLICAR O CÓDIGO DA CÉLULA SEGUINTE PARA A FRENTE RESOLVO ESTE PROBLEMA (string tem whitespace no fim ou início. Python considera string diferente)
#E TAMBÉM O PROBLEMA DE A MESMA PALAVRA ESTAR ESCRITA UMAS VEZES EM MAIÚSCULAS E OUTRAS EM MINÚSCULAS

#First lets look at the 11 values
#X.loc[X['Ocupação Actual_Descrição']=='Desempregada'];

# Comparing to excel, we know that the index 587 (589 in excel) is the one that is missing in our search for "desempregada"
#X.loc[587,:]

# Eliminate white spaces at the begin and end of a string
# X['Ocupação Actual_Descrição']=X['Ocupação Actual_Descrição'].str.strip()
# X['Ocupação Actual_Descrição'].value_counts();

#### Lets eliminate whitespaces and transform all strings in lower key

In [ ]:
# number of unique values before transformations
X.nunique().sort_values(ascending=False)

In [ ]:
# turn all strings lower
X=X.applymap(str.lower);
# eliminate whitespaces
X=X.applymap(str.strip);


In [ ]:
# number of unique values after transformations
X.nunique().sort_values(ascending=False)

In [ ]:
X.describe()

#### Lets try to transform the strings into key words in order to allow us to group information

#### Razão da inscrição e expectativas

In [ ]:
#Procurar palavras mais comuns
teste= [" rise&shine ", " e ", " ou ", " com "]
rgx= '|'.join(teste)
ocup_split=X['Razão da inscrição e expectativas'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()

In [ ]:
condxx=((X['Razão da inscrição e expectativas'].str.contains("descobrir")))#&(X['Trabalho de sonho'].str.contains("trab")))
print((condxx).sum())
#X.loc[condxx,['Razão da inscrição e expectativas']]

#### Ocupação Actual

In [ ]:
teste= [" e ", " ou ", " com "]
rgx= '|'.join(teste)
ocup_split=X['Ocupação Actual'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
# As ocupações mais recorrentes são: desempregada, estudante, trabalhadora, empreendedora, voluntária,
#estagiária. Também aparece "nenhuma" que é igual a desempregada

In [ ]:
#Vamos criar uma nova coluna agrupando as ocupações mais frequentes
i=0;
X['Nova Ocupação Actual']=X['Ocupação Actual']

for k in X['Ocupação Actual']:
    list0=["estudante" in k,"trabalhado" in k, "empreend" in k];
    list1=["estudante" in k,"trabalhado" in k];
    list2=["estudante" in k,"empreend" in k];
    list3=["estudante" in k,"esta" in k];
    list4=["estudante" in k,"volunt" in k];
    list5=["trabalhado" in k,"empreend" in k];
    list6=["trabalhad" in k,"volunt" in k];
    list7=["trabalhador" in k,"estagi" in k];
    list8=["empreende" in k,"volunt" in k];
    list9=["empreende" in k,"estagi" in k];
    list10=["volunt" in k,"estagi" in k];
    list11=["desempreg" in k];
    list12=["estuda" in k];
    list13=["trabalhador" in k];
    list14=["empreen" in k];
    list15=["volunt" in k];
    list16=["estagi" in k];
    if all(list0):
        X['Nova Ocupação Actual'][i]="trabalhadora, estudante e empreendedora"
        i=i+1;        
    elif all(list1):
        X['Nova Ocupação Actual'][i]="trabalhadora e estudante"
        i=i+1;
    elif all(list2):
        X['Nova Ocupação Actual'][i]="empreendedora e estudante"
        i=i+1
    elif all(list3):
        X['Nova Ocupação Actual'][i]="estagiária e estudante"
        i=i+1;
    elif all(list4):
        X['Nova Ocupação Actual'][i]="voluntária e estudante"
        i=i+1;
    elif all(list5):
        X['Nova Ocupação Actual'][i]="trabalhadora e empreendedora"
        i=i+1;
    elif all(list6):
        X['Nova Ocupação Actual'][i]="trabalhadora e voluntária"
        i=i+1;
    elif all(list7):
        X['Nova Ocupação Actual'][i]="trabalhadora e estagiária"
        i=i+1;
    elif all(list8):
        X['Nova Ocupação Actual'][i]="empreendedora e voluntária"
        i=i+1;
    elif all(list9):
        X['Nova Ocupação Actual'][i]="empreendedora e estagiária"
        i=i+1;
    elif all(list10):
        X['Nova Ocupação Actual'][i]="voluntária e estagiária"
        i=i+1;
    elif all(list11):
        X['Nova Ocupação Actual'][i]="desempregada"
        i=i+1;
    elif all(list12):
        X['Nova Ocupação Actual'][i]="estudante"
        i=i+1;
    elif all(list13):
        X['Nova Ocupação Actual'][i]="trabalhadora"
        i=i+1;
    elif all(list14):
        X['Nova Ocupação Actual'][i]="empreendedora"
        i=i+1;
    elif all(list15):
        X['Nova Ocupação Actual'][i]="voluntária"
        i=i+1;
    elif all(list16):
        X['Nova Ocupação Actual'][i]="estagiária"
        i=i+1;             
    else:
        X['Nova Ocupação Actual'][i]="Outro"
        i=i+1
        

In [ ]:
i

In [ ]:
X[['Ocupação Actual', 'Nova Ocupação Actual']].head()

In [ ]:
X['Nova Ocupação Actual'].value_counts()

In [ ]:
# Aumentar a largura das colunas no dataframe
pd.set_option('display.max_colwidth', 260)

In [ ]:
# Ver quais são os outros
cond = (X['Nova Ocupação Actual'] == "Outro");
(X.loc[cond,['Nova Ocupação Actual', 'Ocupação Actual', 'Ocupação Actual_Descrição']])#.groupby('Ocupação Actual')['Nova Ocupação Actual'].count()#.plot.bar()#pie(autopct='%.1f%%', fontsize=16);

In [ ]:
# Vamos incluir os "outros" nos campos já existentes
# Desempregadas
X['Nova Ocupação Actual'][52]="desempregada"
X['Nova Ocupação Actual'][143]="desempregada"
X['Nova Ocupação Actual'][630]="desempregada"

#Trabalhadoras
X['Nova Ocupação Actual'][120]="trabalhadora"
X['Nova Ocupação Actual'][168]="trabalhadora"
X['Nova Ocupação Actual'][469]="trabalhadora"
X['Nova Ocupação Actual'][471]="trabalhadora"
X['Nova Ocupação Actual'][498]="trabalhadora"
X['Nova Ocupação Actual'][499]="trabalhadora"
X['Nova Ocupação Actual'][637]="trabalhadora"
X['Nova Ocupação Actual'][709]="trabalhadora"

#trabalhadora e estudante
X['Nova Ocupação Actual'][163]="trabalhadora e estudante"
X['Nova Ocupação Actual'][516]="trabalhadora e estudante"

#estudante
X['Nova Ocupação Actual'][415]="estudante"

#estagiária
X['Nova Ocupação Actual'][475]="estagiária"
X['Nova Ocupação Actual'][584]="estagiária"

#empreendedora
X['Nova Ocupação Actual'][150]="empreendedora"
X['Nova Ocupação Actual'][926]="empreendedora"

#estudante e voluntária
X['Nova Ocupação Actual'][375]="voluntária e estudante"

In [ ]:
X['Nova Ocupação Actual'].value_counts()

In [ ]:
# Dada a dimensão do dataset (1097 entradas) e tendo em conta que as ocupações "estudante", "trabalhadora",
# "trabalhadora e estudante", "empreendedora" e "desempregada" correpondem a cerca de 93% das entradas,
# vamos eliminar as restantes

In [ ]:
eliminar=X[(X['Nova Ocupação Actual']=="voluntária") | (X['Nova Ocupação Actual']=="voluntária e estudante") |
           (X['Nova Ocupação Actual']=="estagiária e estudante") | (X['Nova Ocupação Actual']=="trabalhadora, estudante e empreendedora") |
            (X['Nova Ocupação Actual']=="empreendedora e estagiária") | (X['Nova Ocupação Actual']=="trabalhadora e empreendedora") |
            (X['Nova Ocupação Actual']=="estagiária") | (X['Nova Ocupação Actual']=="empreendedora e estudante")].index

In [ ]:
X.drop(eliminar,inplace=True)

In [ ]:
X.reset_index(drop=True, inplace=True)

In [ ]:
X['Nova Ocupação Actual'].value_counts()

#### Ocupação Actual_Descrição

In [ ]:
teste= [" e ", " ou ", " com "]
ocup_split=X['Ocupação Actual_Descrição'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

##### Estudantes

In [ ]:
print(((X['Ocupação Actual_Descrição'].str.contains("univer"))&(X['Ocupação Actual_Descrição'].str.contains("mond"))&(X['Nova Ocupação Actual']=="estudante")).sum())

In [ ]:
print(((X['Ocupação Actual_Descrição'].str.contains("univer"))&(X['Ocupação Actual_Descrição'].str.contains("mondlane"))&(X['Nova Ocupação Actual']=="estudante")).sum())

In [ ]:
cond=((X['Ocupação Actual_Descrição'].str.contains("univer"))&(X['Ocupação Actual_Descrição'].str.contains("mond"))&~(X['Ocupação Actual_Descrição'].str.contains("mondlane"))&(X['Nova Ocupação Actual']=="estudante"))
(X.loc[cond,['Nova Ocupação Actual', 'Ocupação Actual', 'Ocupação Actual_Descrição']])

In [ ]:
i=0;
X['Nova Ocupação Actual_Descrição']=X['Ocupação Actual_Descrição']
cnt=0
for k in X['Ocupação Actual_Descrição']:
    list0=["univer" in k,"mond" in k];
    if X['Nova Ocupação Actual'][i]=="estudante":
        if all(list0):
            X['Nova Ocupação Actual_Descrição'][i]="Universidade Eduardo Mondlane"
    i+=1;        

In [ ]:
print(((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante")).sum())

In [ ]:
cond=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
(X.loc[cond,['Nova Ocupação Actual', 'Ocupação Actual', 'Ocupação Actual_Descrição']])['Ocupação Actual_Descrição'].value_counts()

In [ ]:
# Rovuna, católica, zambeze, licungo, lurio ou lúrio, wutive, pedagógica, politécnica, alberto chipande
print('Rovuna')
print(((X['Ocupação Actual_Descrição'].str.contains("univer"))&(X['Ocupação Actual_Descrição'].str.contains("rovu"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante")).sum())

cond1c=((X['Ocupação Actual_Descrição'].str.contains("catolica"))|(X['Ocupação Actual_Descrição'].str.contains("católica")))
cond2c=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
print('católica')
print((cond1c & cond2c).sum())

cond1z=((X['Ocupação Actual_Descrição'].str.contains("zambé"))|(X['Ocupação Actual_Descrição'].str.contains("zambe")))
cond2z=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
print('zambeze')
print((cond1z & cond2z).sum())

cond1l=((X['Ocupação Actual_Descrição'].str.contains("licungo")))#|(X['Ocupação Actual_Descrição'].str.contains("zambe")))
cond2l=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
print('licungo')
print((cond1l & cond2l).sum())

cond1lu=((X['Ocupação Actual_Descrição'].str.contains("lurio"))|(X['Ocupação Actual_Descrição'].str.contains("lúrio")))
cond2lu=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
print('lurio')
print((cond1lu & cond2lu).sum())

cond1w=((X['Ocupação Actual_Descrição'].str.contains("wutive")))#|(X['Ocupação Actual_Descrição'].str.contains("zambe")))
cond2w=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
print('wutive')
print((cond1w & cond2w).sum())

cond1pe=((X['Ocupação Actual_Descrição'].str.contains("pedagó"))|(X['Ocupação Actual_Descrição'].str.contains("pedago")))
cond2pe=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
print('pedagógica maputo')
print((cond1pe & cond2pe).sum())

cond1po=((X['Ocupação Actual_Descrição'].str.contains("polité"))|(X['Ocupação Actual_Descrição'].str.contains("polite")))
cond2po=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
print('politécnica')
print((cond1po & cond2po).sum())

cond1ch=((X['Ocupação Actual_Descrição'].str.contains("chipa")))#|(X['Ocupação Actual_Descrição'].str.contains("polite")))
cond2ch=(~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante"))
print('alberto chipande')
print((cond1ch & cond2ch).sum())



#print(f'Number  : {(((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="estudante")).sum())}')

In [ ]:
# Vamos agrupar as que têm mais de 20 ocorrências num grupo - rovuna, católica, lurio, pedagógica maputo

i=0;
#X['Nova Ocupação Actual_Descrição']=X['Ocupação Actual_Descrição']

for k in X['Ocupação Actual_Descrição']:
    list0=["univer" in k,"mond" in k];
    list1=["univer" in k,"rovu" in k];
    list2=["univer" in k,"católica" in k];
    list3=["univer" in k,"catolica" in k];
    list6=["univer" in k,"lurio" in k];
    list7=["univer" in k,"lúrio" in k];
    list8=["univer" in k,"pedagó" in k];
    list9=["univer" in k,"pedago" in k];


    if X['Nova Ocupação Actual'][i]=="estudante":
        if all(list0):
            X['Nova Ocupação Actual_Descrição'][i]="Universidade Eduardo Mondlane"
        elif all(list1):
            X['Nova Ocupação Actual_Descrição'][i]="Universidade Rovuna"
        elif all(list2) or all(list3):
            X['Nova Ocupação Actual_Descrição'][i]="Universidade Católica"       
        elif all(list6) or all(list7):
            X['Nova Ocupação Actual_Descrição'][i]="Universidade Lúrio"
        elif all(list8) or all(list9):
            X['Nova Ocupação Actual_Descrição'][i]="Universidade Pedagógica"
        else:
            X['Nova Ocupação Actual_Descrição'][i]="Outra Universidade/Não indicada"
    i+=1

In [ ]:
i

In [ ]:
(X.loc[X['Nova Ocupação Actual']=="estudante",['Nova Ocupação Actual_Descrição']]['Nova Ocupação Actual_Descrição']).value_counts()

In [ ]:
plt.figure(figsize=(10,8))
ax=(X.loc[(X['Nova Ocupação Actual']=="estudante"),['Nova Ocupação Actual_Descrição', 'Nova Ocupação Actual']]).groupby('Nova Ocupação Actual_Descrição')['Nova Ocupação Actual'].count().plot(kind='pie',autopct='%.1f%%', fontsize=14)
ax.yaxis.set_visible(False)
# save the figure
plt.savefig('universidades.png', dpi=300)

In [ ]:
plt.figure(figsize=(10,8))
ax=(X.loc[((X['Nova Ocupação Actual']=="estudante") & ~(X['Nova Ocupação Actual_Descrição']=="Outra Universidade/Não indicada")),['Nova Ocupação Actual_Descrição', 'Nova Ocupação Actual']]).groupby('Nova Ocupação Actual_Descrição')['Nova Ocupação Actual'].count().sort_values(ascending=False).plot.pie(autopct='%.1f%%', fontsize=12)
ax.yaxis.set_visible(False)
# save the figure
plt.savefig('universidades_inidivduais.png', dpi=300)

In [ ]:
eliminar=['Universidade Rovuna','Universidade Católica','Universidade Lúrio','Universidade Pedagógica']
X['Nova Ocupação Actual_Descrição']=X['Nova Ocupação Actual_Descrição'].replace(eliminar,'Universidade Rovuna/Católica/Lúrio/Pedagógica')

In [ ]:
#(X.loc[X['Nova Ocupação Actual']=="trabalhadora",['Nova Ocupação Actual_Descrição']]['Nova Ocupação Actual_Descrição']).value_counts()

In [ ]:
plt.figure(figsize=(10,8))
ax=(X.loc[(X['Nova Ocupação Actual']=="estudante"),['Nova Ocupação Actual_Descrição', 'Nova Ocupação Actual']]).groupby('Nova Ocupação Actual_Descrição')['Nova Ocupação Actual'].count().plot(kind='pie',autopct='%.1f%%', fontsize=14)
ax.yaxis.set_visible(False)
# save the figure
plt.savefig('universidades_group.png', dpi=300)

#### Trabalhadoras

In [ ]:
teste= [" de "," e ", " ou ", " com "]
ocup_split=(X.loc[X['Nova Ocupação Actual']=="trabalhadora",['Ocupação Actual_Descrição']]['Ocupação Actual_Descrição']).str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
(X.loc[X['Nova Ocupação Actual']=="trabalhadora",['Ocupação Actual_Descrição']])['Ocupação Actual_Descrição'].value_counts()

In [ ]:
cond1tec=((X['Ocupação Actual_Descrição'].str.contains("tecnica"))|(X['Ocupação Actual_Descrição'].str.contains("técnica")))
cond2tec=((X['Nova Ocupação Actual']=="trabalhadora"))
print('Técnica')
print((cond1tec & cond2tec).sum())

cond1pro=((X['Ocupação Actual_Descrição'].str.contains("profess"))|(X['Ocupação Actual_Descrição'].str.contains("esco"))|(X['Ocupação Actual_Descrição'].str.contains("univer")))
cond2pro=((X['Nova Ocupação Actual']=="trabalhadora"))
print('Professora')
print((cond1pro & cond2pro).sum())

cond1med=((X['Ocupação Actual_Descrição'].str.contains("medic"))|(X['Ocupação Actual_Descrição'].str.contains("médic"))|(X['Ocupação Actual_Descrição'].str.contains("advog")))
cond2med=((X['Nova Ocupação Actual']=="trabalhadora"))
print('Médica/Advogada')
print((cond1med & cond2med).sum())

cond1s=((X['Ocupação Actual_Descrição'].str.contains("secretária"))|(X['Ocupação Actual_Descrição'].str.contains("secretar"))|(X['Ocupação Actual_Descrição'].str.contains("administrat"))|(X['Ocupação Actual_Descrição'].str.contains("assist")))
cond2s=((X['Nova Ocupação Actual']=="trabalhadora"))
print('Secretária/Assistente/Administrativa')
print((cond1s & cond2s).sum())

cond1g=((X['Ocupação Actual_Descrição'].str.contains("gestor")))#|(X['Ocupação Actual_Descrição'].str.contains("médic")))
cond2g=((X['Nova Ocupação Actual']=="trabalhadora"))
print('Gestora')
print((cond1g & cond2g).sum())

cond1v=((X['Ocupação Actual_Descrição'].str.contains("vended"))|(X['Ocupação Actual_Descrição'].str.contains("venda")))
cond2v=((X['Nova Ocupação Actual']=="trabalhadora"))
print('vendedora')
print((cond1v & cond2v).sum())



In [ ]:
# Vamos agrupar as que têm mais de 20 ocorrências num grupo - rovuna, católica, lurio, pedagógica maputo

i=0;
#X['Nova Ocupação Actual_Descrição']=X['Ocupação Actual_Descrição']

for k in X['Ocupação Actual_Descrição']:

    list10=["técnica" in k];
    list11=["tecnica" in k];    
    list12=["profess" in k];
    list13=["esco" in k];
    list14=["univer" in k];
    list15=["medic" in k];
    list16=["médic" in k];
    list17=["advog" in k];
    list18=["secretár" in k];
    list19=["secretar" in k];
    list20=["administrat" in k];
    list21=["assist" in k];
    list22=["gestor" in k];
    list23=["vended" in k];
    list24=["venda" in k];
    
    if X['Nova Ocupação Actual'][i]=="trabalhadora":
        if all(list10) or all(list11):
            X['Nova Ocupação Actual_Descrição'][i]="Técnica"
        elif all(list12) or all(list13) or all(list14):
            X['Nova Ocupação Actual_Descrição'][i]="Professora"
        elif all(list15) or all(list16) or all(list17):
            X['Nova Ocupação Actual_Descrição'][i]="Médica/Advogada"
        elif all(list18) or all(list19) or all(list20) or all(list21):
            X['Nova Ocupação Actual_Descrição'][i]="Secretária/Assistente/Administrativa"
        elif all(list22):
            X['Nova Ocupação Actual_Descrição'][i]="Gestora"
        elif all(list23) or all(list24):
            X['Nova Ocupação Actual_Descrição'][i]="Vendedora"
        else:
            X['Nova Ocupação Actual_Descrição'][i]="Outro Trabalho"

    i=i+1;

In [ ]:
(X.loc[X['Nova Ocupação Actual']=="trabalhadora",['Nova Ocupação Actual_Descrição']]['Nova Ocupação Actual_Descrição']).value_counts()

In [ ]:
# Vanmos incluir as "empreendedoras" na ocupação "trabalhadoras" e dar uma descrição de "tem trabalho próprio"
i=0;

for k in X['Ocupação Actual_Descrição']:
    if X['Nova Ocupação Actual'][i]=="empreendedora":
        X['Nova Ocupação Actual_Descrição'][i]="Tem trabalho próprio"
        X['Nova Ocupação Actual'][i]="trabalhadora"
    i+=1        
    

In [ ]:
plt.figure(figsize=(10,8))
ax=X['Nova Ocupação Actual'].value_counts().plot(kind='pie', autopct='%.1f%%', fontsize=14);
ax.yaxis.set_visible(False)
plt.savefig('Ocupacoes.png', dpi=300)

In [ ]:
(X.loc[X['Nova Ocupação Actual']=="trabalhadora",['Nova Ocupação Actual_Descrição']]['Nova Ocupação Actual_Descrição']).value_counts()

In [ ]:
(X.loc[X['Nova Ocupação Actual']=="trabalhadora",['Nova Ocupação Actual_Descrição', 'Nova Ocupação Actual']]).groupby('Nova Ocupação Actual_Descrição')['Nova Ocupação Actual'].count().sort_values(ascending=False).plot.bar(fontsize=9, figsize=[6,4]);#pie(autopct='%.1f%%', fontsize=16);
plt.savefig('trabalhos.png', dpi=300)


In [ ]:
eliminar=['Gestora','Médica/Advogada','Vendedora', 'Técnica']
X['Nova Ocupação Actual_Descrição']=X['Nova Ocupação Actual_Descrição'].replace(eliminar,'Outro Trabalho')

In [ ]:
(X.loc[X['Nova Ocupação Actual']=="trabalhadora",['Nova Ocupação Actual_Descrição']]['Nova Ocupação Actual_Descrição']).value_counts()

In [ ]:
plt.figure(figsize=(10,8))
ax=(X.loc[X['Nova Ocupação Actual']=="trabalhadora",['Nova Ocupação Actual_Descrição', 'Nova Ocupação Actual']]).groupby('Nova Ocupação Actual_Descrição')['Nova Ocupação Actual'].count().sort_values(ascending=False).plot.pie(autopct='%.1f%%', fontsize=12)
ax.yaxis.set_visible(False)
plt.savefig('trabalho_group.png', dpi=300)

#### Desempregadas

In [ ]:
# Ver se obtemos alguma informação relevante das desempregadas
(X.loc[X['Nova Ocupação Actual']=="desempregada",['Ocupação Actual_Descrição']])['Ocupação Actual_Descrição'].value_counts()

In [ ]:
i=0;

for k in X['Ocupação Actual_Descrição']:
    if X['Nova Ocupação Actual'][i]=="desempregada":
        X['Nova Ocupação Actual_Descrição'][i]="desempregada"
    i+=1        
    

In [ ]:
X['Nova Ocupação Actual'].value_counts()

#### Trabalhadoras e estudantes

In [ ]:
cond1tec=((X['Ocupação Actual_Descrição'].str.contains("tecnica"))|(X['Ocupação Actual_Descrição'].str.contains("técnica")))
cond2tec=((X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('Técnica')
print((cond1tec & cond2tec).sum())

cond1pro=((X['Ocupação Actual_Descrição'].str.contains("profess"))|(X['Ocupação Actual_Descrição'].str.contains("esco"))|(X['Ocupação Actual_Descrição'].str.contains("univer")))
cond2pro=((X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('Professora')
print((cond1pro & cond2pro).sum())

cond1med=((X['Ocupação Actual_Descrição'].str.contains("medic"))|(X['Ocupação Actual_Descrição'].str.contains("médic"))|(X['Ocupação Actual_Descrição'].str.contains("advog")))
cond2med=((X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('Médica/Advogada')
print((cond1med & cond2med).sum())

cond1s=((X['Ocupação Actual_Descrição'].str.contains("secretária"))|(X['Ocupação Actual_Descrição'].str.contains("secretar"))|(X['Ocupação Actual_Descrição'].str.contains("administrat"))|(X['Ocupação Actual_Descrição'].str.contains("assist")))
cond2s=((X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('Secretária/Assistente/Administrativa')
print((cond1s & cond2s).sum())

cond1g=((X['Ocupação Actual_Descrição'].str.contains("gestor")))#|(X['Ocupação Actual_Descrição'].str.contains("médic")))
cond2g=((X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('Gestora')
print((cond1g & cond2g).sum())

cond1v=((X['Ocupação Actual_Descrição'].str.contains("vended"))|(X['Ocupação Actual_Descrição'].str.contains("venda")))
cond2v=((X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('vendedora')
print((cond1v & cond2v).sum())


In [ ]:
print('Eduardo Mondlane')
print(((X['Ocupação Actual_Descrição'].str.contains("univer"))&(X['Ocupação Actual_Descrição'].str.contains("mond"))&(X['Nova Ocupação Actual']=="trabalhadora e estudante")).sum())

print('Rovuna')
print(((X['Ocupação Actual_Descrição'].str.contains("univer"))&(X['Ocupação Actual_Descrição'].str.contains("rovu"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante")).sum())

cond1c=((X['Ocupação Actual_Descrição'].str.contains("catolica"))|(X['Ocupação Actual_Descrição'].str.contains("católica")))
cond2c=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('católica')
print((cond1c & cond2c).sum())

cond1z=((X['Ocupação Actual_Descrição'].str.contains("zambé"))|(X['Ocupação Actual_Descrição'].str.contains("zambe")))
cond2z=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('zambeze')
print((cond1z & cond2z).sum())

cond1l=((X['Ocupação Actual_Descrição'].str.contains("licungo")))#|(X['Ocupação Actual_Descrição'].str.contains("zambe")))
cond2l=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('licungo')
print((cond1l & cond2l).sum())

cond1lu=((X['Ocupação Actual_Descrição'].str.contains("lurio"))|(X['Ocupação Actual_Descrição'].str.contains("lúrio")))
cond2lu=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('lurio')
print((cond1lu & cond2lu).sum())

cond1w=((X['Ocupação Actual_Descrição'].str.contains("wutive")))#|(X['Ocupação Actual_Descrição'].str.contains("zambe")))
cond2w=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('wutive')
print((cond1w & cond2w).sum())

cond1pe=((X['Ocupação Actual_Descrição'].str.contains("pedagó"))|(X['Ocupação Actual_Descrição'].str.contains("pedago")))
cond2pe=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('pedagógica maputo')
print((cond1pe & cond2pe).sum())

cond1po=((X['Ocupação Actual_Descrição'].str.contains("polité"))|(X['Ocupação Actual_Descrição'].str.contains("polite")))
cond2po=((X['Ocupação Actual_Descrição'].str.contains("univer"))&~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('politécnica')
print((cond1po & cond2po).sum())

cond1ch=((X['Ocupação Actual_Descrição'].str.contains("chipa")))#|(X['Ocupação Actual_Descrição'].str.contains("polite")))
cond2ch=(~(X['Nova Ocupação Actual_Descrição']=="Universidade Eduardo Mondlane")&(X['Nova Ocupação Actual']=="trabalhadora e estudante"))
print('alberto chipande')
print((cond1ch & cond2ch).sum())


#### poucas responderam a universidade onde estudam. Vamos apenas indicar onde trabalham

In [ ]:
# Vamos agrupar as que têm mais de 20 ocorrências num grupo - rovuna, católica, lurio, pedagógica maputo

i=0;
#X['Nova Ocupação Actual_Descrição']=X['Ocupação Actual_Descrição']

for k in X['Ocupação Actual_Descrição']:

    list10=["técnica" in k];
    list11=["tecnica" in k];    
    list12=["profess" in k];
    list13=["esco" in k];
    list14=["univer" in k];
    list15=["medic" in k];
    list16=["médic" in k];
    list17=["advog" in k];
    list18=["secretár" in k];
    list19=["secretar" in k];
    list20=["administrat" in k];
    list21=["assist" in k];
    list22=["gestor" in k];
    list23=["vended" in k];
    list24=["venda" in k];
    
    if X['Nova Ocupação Actual'][i]=="trabalhadora e estudante":
        if all(list10) or all(list11):
            X['Nova Ocupação Actual_Descrição'][i]="Técnica"
        elif all(list12) or all(list13) or all(list14):
            X['Nova Ocupação Actual_Descrição'][i]="Professora"
        elif all(list15) or all(list16) or all(list17):
            X['Nova Ocupação Actual_Descrição'][i]="Médica/Advogada"
        elif all(list18) or all(list19) or all(list20) or all(list21):
            X['Nova Ocupação Actual_Descrição'][i]="Secretária/Assistente/Administrativa"
        elif all(list22):
            X['Nova Ocupação Actual_Descrição'][i]="Gestora"
        elif all(list23) or all(list24):
            X['Nova Ocupação Actual_Descrição'][i]="Vendedora"
        else:
            X['Nova Ocupação Actual_Descrição'][i]="Outro Trabalho"

    i=i+1;

In [ ]:
(X.loc[X['Nova Ocupação Actual']=="trabalhadora e estudante",['Nova Ocupação Actual_Descrição']]['Nova Ocupação Actual_Descrição']).value_counts()

In [ ]:
eliminar=['Gestora','Médica/Advogada','Vendedora','Técnica']
X['Nova Ocupação Actual_Descrição']=X['Nova Ocupação Actual_Descrição'].replace(eliminar,'Outro Trabalho')

In [ ]:
plt.figure(figsize=(10,8))
ax=(X.loc[X['Nova Ocupação Actual']=="trabalhadora e estudante",['Nova Ocupação Actual_Descrição', 'Nova Ocupação Actual']]).groupby('Nova Ocupação Actual_Descrição')['Nova Ocupação Actual'].count().sort_values(ascending=False).plot.pie(autopct='%.1f%%', fontsize=12)
ax.yaxis.set_visible(False)
plt.savefig('trabalha e estuda.png', dpi=300)

In [ ]:
X.describe()

#### Trabalho de sonho

In [ ]:
X.columns

In [ ]:
#ocup_split=X['Trabalho de sonho'].str.replace(rgx,' ')

In [ ]:
teste= [" meu ","de "," é "," pois "," das "," eu "," trabalhar ", " numa ","gostaria ", " porque ", " uma "," de ", " e ", " que ", " a ", " o ", " na ", " com ", " um ", " é ", " as ", " os ", " em ", " para ", " da ", " do ", " no ", " ou "]
rgx= '|'.join(teste)

ocup_split=X['Trabalho de sonho'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
cond7=((X['Trabalho de sonho'].str.contains("minha"))&(X['Trabalho de sonho'].str.contains("empresa")))
cond8=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
cond6=((X['Trabalho de sonho'].str.contains("próp"))&(X['Trabalho de sonho'].str.contains("emp")))
cond5=((X['Trabalho de sonho'].str.contains("próp"))&(X['Trabalho de sonho'].str.contains("proj")))
print('meu negócio')
print((cond7 | cond8 | cond6| cond5).sum())

cond9=((X['Trabalho de sonho'].str.contains("ong"))|(X['Trabalho de sonho'].str.contains("organização não"))|(X['Trabalho de sonho'].str.contains("save the"))|(X['Trabalho de sonho'].str.contains("unicef"))|(X['Trabalho de sonho'].str.contains("onu"))|(X['Trabalho de sonho'].str.contains("direitos"))|(X['Trabalho de sonho'].str.contains("humanitár"))|(X['Trabalho de sonho'].str.contains("nações")))
#cond10=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
print('ONG')
print((cond9).sum())

cond10=((X['Trabalho de sonho'].str.contains("banco"))|(X['Trabalho de sonho'].str.contains("bancá")))
#cond10=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
print('Trabalhar num banco')
print((cond10).sum())

cond11=((X['Trabalho de sonho'].str.contains("escola")))#|(X['Trabalho de sonho'].str.contains("profes")))
#cond10=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
print('Numa escola')
print((cond11).sum())

cond12=((X['Trabalho de sonho'].str.contains("saúde"))|(X['Trabalho de sonho'].str.contains("hospita"))|(X['Trabalho de sonho'].str.contains("médic")))
#cond10=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
print('Saúde/Hospitalar')
print((cond12).sum())

cond13=((X['Trabalho de sonho'].str.contains("ministé")))#|(X['Trabalho de sonho'].str.contains("organização não")))
#cond10=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
print('ministério')
print((cond13).sum())

In [ ]:
i=0;
X['Nova Trabalho de sonho']=X['Trabalho de sonho']

for k in X['Trabalho de sonho']:
    list0=["minha" in k,"empresa" in k];
    list1=["meu" in k,"negóc" in k];
    list2=["próp" in k,"emp" in k];
    
    list3=["ong" in k];
    list4=["organiza" in k, "governam" in k];
    list5=["save the" in k];
    list6=["unicef" in k];
    list7=["onu" in k];
    list8=["direitos" in k];
    list9=["humanitár" in k];
    list10=["nações" in k];
    
    list11=["banco" in k];
    list12=["bancá" in k];
    
    list13=["escola" in k];
    
    list14=["saúde" in k];
    list15=["hospita" in k];
    list16=["médic" in k];
    
    list17=["ministé" in k];
    
    list18=["empresa" in k];
    

    if all(list0) or all(list1) or all(list2):
        X['Nova Trabalho de sonho'][i]="Ter próprio negócio"
    elif all(list3) or all(list4) or all(list5) or all(list6) or all(list7) or all(list8) or all(list9) or all(list10):
        X['Nova Trabalho de sonho'][i]="Ter trabalho humanitário (ONG)"
    elif all(list11) or all(list12):
        X['Nova Trabalho de sonho'][i]="Trabalhar num banco"
    elif all(list13):
        X['Nova Trabalho de sonho'][i]="Trabalhar numa escola"
    elif all(list14) or all(list15) or all(list16):
        X['Nova Trabalho de sonho'][i]="Trabalhar área saúde"
    elif all(list17):
        X['Nova Trabalho de sonho'][i]="Trabalhar num ministério"
    elif all(list18):
        X['Nova Trabalho de sonho'][i]="Trabalhar numa empresa"
    else:
        X['Nova Trabalho de sonho'][i]="Outro sonho"

    i=i+1;

In [ ]:
X['Nova Trabalho de sonho'].value_counts()

In [ ]:
plt.figure(figsize=(10,8))
ax=X['Nova Trabalho de sonho'].value_counts().plot(kind='pie', autopct='%.1f%%', fontsize=14);
ax.yaxis.set_visible(False)
plt.savefig('Trabalho sonho.png', dpi=300)

In [ ]:
plt.figure(figsize=(10,8))
ax=(X.loc[(~(X['Nova Trabalho de sonho']=="Outro sonho")),['Nova Trabalho de sonho']])['Nova Trabalho de sonho'].value_counts().plot.pie(autopct='%.1f%%', fontsize=12)
ax.yaxis.set_visible(False)
# save the figure
plt.savefig('Trabalho sonho individuais.png', dpi=300)

In [ ]:
eliminar=['Trabalhar numa escola','Trabalhar num ministério', 'Trabalhar área saúde', 'Trabalhar num banco']
X['Nova Trabalho de sonho']=X['Nova Trabalho de sonho'].replace(eliminar,'Outro sonho')

In [ ]:
X['Nova Trabalho de sonho'].value_counts()

In [ ]:
plt.figure(figsize=(10,8))
ax=X['Nova Trabalho de sonho'].value_counts().plot(kind='pie', autopct='%.1f%%', fontsize=14);
ax.yaxis.set_visible(False)
plt.savefig('Trabalho sonho_group.png', dpi=300)

In [ ]:
condxx=((X['Trabalho de sonho'].str.contains("")))#&(X['Trabalho de sonho'].str.contains("trab")))
X.loc[condxx,['Trabalho de sonho']]

#### Próximos Passos

In [ ]:
ocup_split=X['Próximos Passos'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
i=0;
X['Nova Próximos Passos']=X['Próximos Passos']

for k in X['Próximos Passos']:
    list0=["minha" in k,"empresa" in k];
    list1=["meu" in k,"negóc" in k];
    list2=["próp" in k,"emp" in k];
    list21=["meu" in k,"negoc" in k];
    
    list3=["ong" in k];
    list4=["organiza" in k, "governam" in k];
    list5=["save the" in k];
    list6=["unicef" in k];
    list7=["onu" in k];
    list8=["direitos" in k];
    list9=["humanitár" in k];
    list10=["nações" in k];
    
    list11=["mestrado" in k];
    list12=["curso" in k];
    list13=["estud" in k];
    list133=["licen" in k];
    
    list14=["trabalhar" in k, "numa" in k, "empresa" in k];
    

    if all(list0) or all(list1) or all(list2) or all(list21):
        X['Nova Próximos Passos'][i]="Ter próprio negócio"
    elif all(list3) or all(list4) or all(list5) or all(list6) or all(list7) or all(list8) or all(list9) or all(list10):
        X['Nova Próximos Passos'][i]="Ter trabalho humanitário (ONG)"
    elif all(list11) or all(list12) or all(list13) or all(list133):
        X['Nova Próximos Passos'][i]="Estudar"
    elif all(list14):
        X['Nova Próximos Passos'][i]="Trabalhar numa empresa"
    else:
        X['Nova Próximos Passos'][i]="Outros"
    i+=1

In [ ]:
X['Nova Próximos Passos'].value_counts()

In [ ]:
figsize=(10,8)
X['Nova Próximos Passos'].value_counts().plot(kind='bar', fontsize=10);
plt.savefig('próximos passos.png', dpi=300)

In [ ]:
#vamos eliminar os outros
elimina=X[(X['Nova Próximos Passos']=='Outros')].index

In [ ]:
elimina

In [ ]:
X.drop(elimina,inplace=True)
X.reset_index(drop=True, inplace=True)

In [ ]:
plt.figure(figsize=(10,8))
ax=X['Nova Próximos Passos'].value_counts().plot(kind='pie', autopct='%.1f%%', fontsize=14);
ax.yaxis.set_visible(False)
plt.savefig('próximos passos_group.png', dpi=300)


In [ ]:
X.describe()

#### Apenas uma coisa para fazer durante o resto da vida

In [ ]:
ocup_split=X['Apenas uma coisa para fazer durante o resto da vida'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
cond11=((X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("ajudar"))|(X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("comunid"))|(X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("crianç"))|(X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("idos"))|(X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("ong"))|(X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("humani"))|(X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("cuida")))#|(X['Trabalho de sonho'].str.contains("profes")))
#cond10=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
print('ajudar')
print((cond11).sum())

cond11=((X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("feliz")))#|(X['Trabalho de sonho'].str.contains("profes")))
#cond10=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
print('feli')
print((cond11).sum())

cond11=((X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("empresá"))|(X['Apenas uma coisa para fazer durante o resto da vida'].str.contains("empreen")))#|(X['Trabalho de sonho'].str.contains("profes")))
#cond10=((X['Trabalho de sonho'].str.contains("meu"))&(X['Trabalho de sonho'].str.contains("neg")))
print('empresária/empreendedora')
print((cond11).sum())


In [ ]:
i=0;
X['Nova Apenas uma coisa para fazer durante o resto da vida']=X['Apenas uma coisa para fazer durante o resto da vida']

for k in X['Apenas uma coisa para fazer durante o resto da vida']:
    list0=["ajudar" in k];
    list1=["cuidar" in k];
    list2=["ong" in k];
    list3=["crianç" in k];
    list4=["comunid" in k];
    list5=["idosos" in k];
    list6=["humanit" in k];
    list7=["nações" in k];
    list8=["save the" in k];
    list9=["unicef" in k];
    list10=["organiza" in k, "governam" in k];
    
    list11=["feliz" in k];
    
    list12=["empresá" in k];
    list13=["empreen" in k];
    

    if all(list0) or all(list1) or all(list2) or all(list3) or all(list4) or all(list5) or all(list6) or all(list7) or all(list8) or all(list9) or all(list10):
        X['Nova Apenas uma coisa para fazer durante o resto da vida'][i]="Ajudar"
    elif all(list11):
        X['Nova Apenas uma coisa para fazer durante o resto da vida'][i]="Ser feliz"
    elif all(list12) or all(list13):
        X['Nova Apenas uma coisa para fazer durante o resto da vida'][i]="Ser empresária/empreendedora"
    else:
        X['Nova Apenas uma coisa para fazer durante o resto da vida'][i]="Outra"
    i+=1

In [ ]:
X['Nova Apenas uma coisa para fazer durante o resto da vida'].value_counts()

In [ ]:
plt.figure(figsize=(10,8))
ax=X['Nova Apenas uma coisa para fazer durante o resto da vida'].value_counts().plot(kind='pie', autopct='%.1f%%', fontsize=14);
ax.yaxis.set_visible(False)
plt.savefig('Apenas uma coisa.png', dpi=300)


#### Maior ambição

In [ ]:
ocup_split=X['Maior ambição'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
i=0;
X['Nova Maior ambição']=X['Maior ambição']

for k in X['Maior ambição']:
    list0=["minha" in k,"empresa" in k];
    list1=["meu" in k,"negóc" in k];
    list2=["próp" in k,"emp" in k];
    list3=["ter" in k,"empres" in k];
    
    list4=["ong" in k];
    list5=["organiza" in k, "governam" in k];
    list6=["save the" in k];
    list7=["unicef" in k];
    list8=["onu" in k];
    list9=["direitos" in k];
    list10=["humanitár" in k];
    list11=["nações" in k];
    
    list14=["mestrado" in k];
    list12=["curso" in k];
    list13=["estud" in k];
    
    list15=["ajudar" in k];
    
    list16=["feliz" in k];
    

    if all(list0) or all(list1) or all(list2) or all(list3):
        X['Nova Maior ambição'][i]="Ter próprio negócio"
    elif all(list11) or all(list4) or all(list5) or all(list6) or all(list7) or all(list8) or all(list9) or all(list10):
        X['Nova Maior ambição'][i]="Ter trabalho humanitário (ONG)"
    elif all(list14) or all(list12) or all(list13):
        X['Nova Maior ambição'][i]="Estudar"
    elif all(list15):
        X['Nova Maior ambição'][i]="Ajudar"
    elif all(list16):
        X['Nova Maior ambição'][i]="Feliz"
    else:
        X['Nova Maior ambição'][i]="Outros"
    i+=1

In [ ]:
X['Nova Maior ambição'].value_counts()

In [ ]:
X['Nova Maior ambição'].value_counts().plot(kind='bar', fontsize=14);

In [ ]:
X.describe()

#### Clustering

In [ ]:
Z=X.copy()

In [ ]:
Z.columns

In [ ]:
Z.drop(columns=['Razão da inscrição e expectativas',
                'Ocupação Actual',
                'Ocupação Actual_Descrição',
                'Próximos Passos',
                'Trabalho de sonho',
                'Maior ambição',
                'Apenas uma coisa para fazer durante o resto da vida'], inplace=True)

In [ ]:
Z['Tratamento de dados'].value_counts()

In [ ]:
Z.drop(columns=['Tratamento de dados'], inplace=True)

In [ ]:
Z.drop(columns=['Nova Maior ambição'], inplace=True)

In [ ]:
Z.describe()

In [ ]:
# Elbow curve to find optimal K
cost = []
K = range(1,20)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 30, verbose=1)
    kmode.fit_predict(Z)
    cost.append(kmode.cost_)
    
plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
km = KModes(n_clusters=11, init='Huang', n_init=100, verbose=1)

clusters = km.fit_predict(Z)

# Print the cluster centroids
print(km.cluster_centroids_)

In [ ]:
# Add the cluster to the dataframe
Z['Cluster Labels'] = km.labels_
Z['Cluster Labels'].value_counts()

In [ ]:
km.cluster_centroids_[0]

In [ ]:
Z.head(2)

### OHE

In [ ]:
#Y=X.copy()

In [ ]:
#Y.drop(columns=['Razão da inscrição e expectativas',
 #               'Ocupação Actual',
  #              'Ocupação Actual_Descrição',
   #             'Próximos Passos',
    #            'Trabalho de sonho',
     #           'Maior ambição',
      #          'Apenas uma coisa para fazer durante o resto da vida'], inplace=True)

In [ ]:
#Y.drop(columns=['Tratamento de dados'], inplace=True)

In [ ]:
#Y.drop(columns=['Nova Maior ambição'], inplace=True)

In [ ]:
#Y.describe()

In [ ]:
#Y=pd.get_dummies(Y, prefix='Já conhecia a GirlMove', columns=['Já conhecia a GirlMOVE Academy'])

In [ ]:
#Y=pd.get_dummies(Y, prefix='Ocupação', columns=['Nova Ocupação Actual'])

In [ ]:
#Y=pd.get_dummies(Y, prefix='Ocupação Descrição', columns=['Nova Ocupação Actual_Descrição'])

In [ ]:
#Y=pd.get_dummies(Y, prefix='Trabalho sonho', columns=['Nova Trabalho de sonho'])

In [ ]:
#Y=pd.get_dummies(Y, prefix='Próximos Passos', columns=['Nova Próximos Passos'])

In [ ]:
#Y.head()

In [ ]:
#Y.columns[1]

In [ ]:
#i=1
#j=0
#Y['cluster']=''
#for i in range (0,len(Y),1):
 #   for j in range (0,(Y.shape[1]-1),1):
  #      if Y.iloc[i,j]==1:
   #         Y['cluster'][i]=Y['cluster'][i] + ' , ' + (Y.columns[j])
    #    j+=1
    #i+=1
    

In [ ]:
#Y['cluster'].value_counts()

## Ficheiro "Feedback Rise" (rise)

In [ ]:
rise.shape

In [ ]:
rise.dtypes

In [ ]:
rise.nunique().sort_values(ascending=False)

In [ ]:
rise.isnull().sum()

In [ ]:
rise.head(2)

In [ ]:
rise.describe()

In [ ]:
plt.figure(figsize=(16,14))
i=1
for k in rise.select_dtypes(exclude=object).columns:
    mean=rise[k].mean()
    std=rise[k].std()
    plt.subplot(2, 3, i)
    #plt.title(k)
    rise[k].value_counts().plot(kind='bar')
    plt.xlabel(k)
    i+=1

In [ ]:
print(f'média avaliação Plataforma =' ,rise['Como avalias a plataforma?'].mean())
print(f'média avaliação "Celebrar" =' ,rise['O capítulo "Celebrar" fez-te refletir e foi útil?'].mean())
print(f'média avaliação "Ikigai" =' ,rise['O capítulo "Ikigai" transmitiu-te novas aprendizagens?'].mean())
print(f'média avaliação "Recursos extra" =' ,rise['Os recursos extra foram úteis? '].mean())

In [ ]:
# turn all strings lower
rise['Deixa-nos um comentário sobre o módulo Rise.']=rise['Deixa-nos um comentário sobre o módulo Rise.'].str.lower();
# eliminate whitespaces
rise['Deixa-nos um comentário sobre o módulo Rise.']=rise['Deixa-nos um comentário sobre o módulo Rise.'].str.strip();


In [ ]:
teste= [" o ", " a "," meu ","de "," é "," pois "," das "," eu "," trabalhar ", " numa ","gostaria ", " porque ", " uma "," de ", " e ", " que ", " a ", " o ", " na ", " com ", " um ", " é ", " as ", " os ", " em ", " para ", " da ", " do ", " no ", " ou "]
rgx= '|'.join(teste)
ocup_split=rise['Deixa-nos um comentário sobre o módulo Rise.'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
cond1=((rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("ikigai"))&(rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("gost")))
cond2=((rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("gost"))&(rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("celebr")))
cond3=((rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("gost"))&(rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("menos")))

print('gostei ikigai')
print((cond1).sum())
print('gostei celebrar')
print((cond2).sum())
print('gostei menos')
print((cond3).sum())

cond4=((rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("ikigai"))&(rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("aprend")))
cond5=((rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("aprend"))&(rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("celebr")))

print('aprender ikigai')
print((cond4).sum())
print('aprender celebrar')
print((cond5).sum())

In [ ]:
rise.loc[cond3, ['Deixa-nos um comentário sobre o módulo Rise.']]

In [ ]:
print(rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("aprendi").sum())

In [ ]:
cond1 = ((rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("aprend")))# & (rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("trabalhadora")))
#cond2 = ((rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("trab")) & (rise['Deixa-nos um comentário sobre o módulo Rise.'].str.contains("est")))

jw=(rise.loc[cond1,['Deixa-nos um comentário sobre o módulo Rise.']])

## Ficheiro "Feedback Shine" (shine)

In [ ]:
shine.head(2)

In [ ]:
shine.shape

In [ ]:
shine.dtypes

In [ ]:
shine.isnull().sum()

In [ ]:
shine.nunique()

In [ ]:
shine.describe()

In [ ]:
# turn all strings lower
shine['Qual a principal aprendizagem do módulo Shine?']=shine['Qual a principal aprendizagem do módulo Shine?'].str.lower();
# eliminate whitespaces
shine['Qual a principal aprendizagem do módulo Shine?']=shine['Qual a principal aprendizagem do módulo Shine?'].str.strip();


In [ ]:
teste= [" shine ", " mas "," o ", " a "," meu ","de "," é "," pois "," das "," eu ", " numa ","gostaria ", " porque ", " uma "," de ", " e ", " que ", " a ", " o ", " na ", " com ", " um ", " é ", " as ", " os ", " em ", " para ", " da ", " do ", " no ", " ou "]
rgx= '|'.join(teste)
ocup_split=shine['Qual a principal aprendizagem do módulo Shine?'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
cond1=((shine['Qual a principal aprendizagem do módulo Shine?'].str.contains("diferença"))|(shine['Qual a principal aprendizagem do módulo Shine?'].str.contains("manas"))|(shine['Qual a principal aprendizagem do módulo Shine?'].str.contains("influ"))|(shine['Qual a principal aprendizagem do módulo Shine?'].str.contains("espalha"))|(shine['Qual a principal aprendizagem do módulo Shine?'].str.contains("inspirar")))

print('Inspirar outros')
print((cond1).sum())

In [ ]:
i=0
shine['Aprendizagem']=shine['Qual a principal aprendizagem do módulo Shine?']

for k in shine['Qual a principal aprendizagem do módulo Shine?']:
    list0=['diferença' in k];
    list1=['manas' in k];
    list2=['influ' in k];
    list3=['espalha' in k];
    list4=['inspirar' in k];
    if all(list0) or all(list1) or all(list2) or all(list3) or all(list4):
        shine['Aprendizagem'][i]='Inspirar Outras Pessoas'
    else:
        shine['Aprendizagem'][i]='Outra aprendizagem'
    i+=1

In [ ]:
i

In [ ]:
ax=shine['Aprendizagem'].value_counts().plot(kind='pie',autopct='%.1f%%', fontsize=16, figsize=(10,6));
ax.yaxis.set_visible(False)
plt.savefig('Aprendizagem.png', dpi=300)

In [ ]:
shine.loc[cond1,['Qual a principal aprendizagem do módulo Shine?']]

In [ ]:
teste= [" shine ", " mas "," o ", " a "," meu ","de "," é "," pois "," das "," eu ", " numa ","gostaria ", " porque ", " uma "," de ", " e ", " que ", " a ", " o ", " na ", " com ", " um ", " é ", " as ", " os ", " em ", " para ", " da ", " do ", " no ", " ou "]
rgx= '|'.join(teste)
ocup_split=shine['Deixa-nos um comentário sobre o módulo Shine.'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
cond1=((shine['Deixa-nos um comentário sobre o módulo Shine.'].str.contains("dific"))|(shine['Deixa-nos um comentário sobre o módulo Shine.'].str.contains("difíc")))#|(shine['Qual a principal aprendizagem do módulo Shine?'].str.contains("influen"))|(shine['Qual a principal aprendizagem do módulo Shine?'].str.contains("espalha"))|(shine['Qual a principal aprendizagem do módulo Shine?'].str.contains("inspirar")))

print('dificuldade')
print((cond1).sum())

In [ ]:
shine.loc[cond1,['Deixa-nos um comentário sobre o módulo Shine.']]

In [ ]:
shine.describe()

## Ficheiro "Relatório impacto" (rel_impact)

In [ ]:
rel_impact=pd.read_excel('Relatório de Impacto (Respostas).xlsx')

In [ ]:
rel_impact.dtypes

In [ ]:
rel_impact=rel_impact.drop(columns=['Idade'])
rel_impact=rel_impact.astype(str)
# turn all strings lower
rel_impact=rel_impact.applymap(str.lower);
# eliminate whitespaces
rel_impact=rel_impact.applymap(str.strip);

In [ ]:
rel_impact.isnull().sum()

In [ ]:
rel_impact['Tiveram atividades para a comunidade?'].value_counts()

In [ ]:
rel_impact=rel_impact.replace(np.nan, '')#(value='', inplace=True)

In [ ]:
rel_impact['Tiveram atividades para a comunidade?'].value_counts()

In [ ]:
rel_impact['novo_ac']=rel_impact['Tiveram atividades para a comunidade?']+rel_impact['Tiveram atividades para a comunidade?.1']

In [ ]:

rel_impact['novo']= ((rel_impact['Que atividades desenvolveste?']) + (rel_impact['Que atividades desenvolveram?']))

In [ ]:
rel_impact['novo'];

In [ ]:
teste= ["como "," elas ", "que"," mas "," o ", " a "," meu ","de "," é "," pois "," das "," eu ", " numa ", " uma "," de ", " e ", " que ", " a ", " o ", " na ", " com ", " um ", " é ", " as ", " os ", " em ", " para ", " da ", " do ", " no ", " ou "]
rgx= '|'.join(teste)
ocup_split=rel_impact['novo'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
rel_impact.dtypes

In [ ]:
#sisterhoodcircle sozinha vs em par
rel_impact['Fizeste acontecer o sisterhoodcircle sozinha ou com outra mana?'].value_counts().plot(kind='bar');

In [ ]:
teste= [" mas "," o ", " a "," meu ","de "," é "," pois "," das "," eu ", " numa ","gostaria ", " porque ", " uma "," de ", " e ", " que ", " a ", " o ", " na ", " com ", " um ", " é ", " as ", " os ", " em ", " para ", " da ", " do ", " no ", " ou "]
rgx= '|'.join(teste)
ocup_split=rel_impact['Que atividades desenvolveste?'].str.replace(rgx,' ')
ocup_split=ocup_split.str.split(' ', expand=True)
ocup_split
novo=ocup_split.stack()
novo.value_counts()
#novo.str.contains("studante|rabalhadora").sum()
#ocup_split

In [ ]:
rel_impact.nunique()

# Só deveriam existir 2 respostas à pergunta 5. Resolver

### Análise Extra

In [ ]:
X.describe()

In [ ]:
table = pd.crosstab(X['Nova Ocupação Actual'], X['Nova Próximos Passos'])
display(table)
chi2, pval, dof, expected=stats.chi2_contingency(table)
print(expected)
print(pval)

O chi-square test entre os próximos passos das candidatas e a sua ocupação actual mostra que a relação entre estas 2 variáveis é estatisticamente significativa (p-value <0.05). Podemos assim dizer que as 2 variáveis estão relacionadas. 

In [ ]:
table.plot(kind="bar", figsize=(10,6), fontsize=12);#,stacked=True);